In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Torch modules for the autoencoders
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [4]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [5]:
def to_img(x):
    x = 0.5*(x+1)
    x = x.clamp(0,1)
    x = x.view(x.size(0), 1,28,28)
    return x
        

In [6]:
#Defining some hyperparameters
num_epochs = 100
batch_size = 128
learning_rate = 0.001


In [7]:
# Defining the image transforms
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [8]:
# Loading data
dataset = MNIST('./data', transform=transform, download = True)


In [9]:
dataset

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None

In [10]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataloader

In [11]:
#plt.imshow(dataloader)

In [12]:
# Defining Encoder and Decoder
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28,128),
            nn.ReLU(True),
            nn.Linear(128,64),
            nn.ReLU(True),
            nn.Linear(64,12),
            nn.ReLU(True),
            nn.Linear(12,3))
        
        self.decoder = nn.Sequential(
            nn.Linear(3,12),
            nn.ReLU(True),
            nn.Linear(12,64),
            nn.ReLU(True),
            nn.Linear(64,128),
            nn.ReLU(True),
            nn.Linear(128,28*28),
            nn.Tanh())
        # Defininf the forward feed
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [13]:
# Deining model
model = autoencoder().cuda()
print(model)

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=64, out_features=12, bias=True)
    (5): ReLU(inplace)
    (6): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
)


In [14]:
# Definining the optimizer and the loss criterion
criterion= nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=1e-5)

In [15]:
# traing the model
for epoch in range(num_epochs):
    for data in dataloader:
        img,_ = data
        #print('img', img)
        #print('_', _)
        img = img.view(img.size(0), -1)
        #print(img)
        img = Variable(img).cuda()
        #print(img)
    ## Forward pass##
        #print(model)
        output = model(img)
       
        loss = criterion(output, img)
    
    ## backward ##
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    #log#
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.2023
epoch [2/100], loss:0.1733
epoch [3/100], loss:0.1803
epoch [4/100], loss:0.1566
epoch [5/100], loss:0.1482
epoch [6/100], loss:0.1424
epoch [7/100], loss:0.1472
epoch [8/100], loss:0.1380
epoch [9/100], loss:0.1424
epoch [10/100], loss:0.1398
epoch [11/100], loss:0.1423
epoch [12/100], loss:0.1209
epoch [13/100], loss:0.1315
epoch [14/100], loss:0.1340
epoch [15/100], loss:0.1317
epoch [16/100], loss:0.1346
epoch [17/100], loss:0.1317
epoch [18/100], loss:0.1382
epoch [19/100], loss:0.1372
epoch [20/100], loss:0.1245
epoch [21/100], loss:0.1239
epoch [22/100], loss:0.1248
epoch [23/100], loss:0.1167
epoch [24/100], loss:0.1276
epoch [25/100], loss:0.1252
epoch [26/100], loss:0.1276
epoch [27/100], loss:0.1397
epoch [28/100], loss:0.1355
epoch [29/100], loss:0.1172
epoch [30/100], loss:0.1207
epoch [31/100], loss:0.1222
epoch [32/100], loss:0.1293
epoch [33/100], loss:0.1175
epoch [34/100], loss:0.1187
epoch [35/100], loss:0.1203
epoch [36/100], loss:0.1251
e

In [16]:
print(os.getcwd())

D:\1st year\Python codes\CIFR Analysis\self contained analysis
